### In this notebook, we show how to retrieve the Kasthuri graph

- Our graph is derived from RAMON Metadata, and so may differ slightly from that of the graph in the paper.  
- For a small percentage (2%) of the connections, we reassigned either neuron or synapse labels based on geometries 
 and paint information.


In [9]:
import numpy as np
import ndio.remote.neurodata as neurodata
import ndio.ramon as ramon
import time
import ndio

start = time.time()

token = 'kasthuri2015_ramon_v3'
channel = 'synapses'
res = 3

In [10]:
# Find all synapses

nd = neurodata()

id_synapse = nd.get_ramon_ids(token, channel, ramon_type=ramon.RAMONSynapse)
len(id_synapse)

1700

In [8]:
#for x in range(np.shape(id_synapse)[0]):
x = 2
s = nd.get_ramon_metadata(token,channel,[id_synapse[x]])
print s

KeyError: "Unable to open object (Object 'synapsetype' doesn't exist)"